# Классификация на PyTorch


## Импорт библиотек

Подключаем необходимые библиотеки:
- `torch` - основная библиотека PyTorch
- `torch.nn` - модуль для создания нейронных сетей
- `torch.optim` - оптимизаторы для обучения
- `Dataset, DataLoader` - для работы с данными
- `sklearn` - для загрузки датасета и метрик


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


## Загрузка и подготовка данных

Используем датасет Wine - классификация вин на 3 класса по 13 химическим признакам:
1. Загружаем данные и разделяем на train/test
2. Нормализуем признаки с помощью `StandardScaler` (важно для обучения нейросетей)
3. Конвертируем numpy массивы в PyTorch тензоры
   - `FloatTensor` для признаков (X)
   - `LongTensor` для меток классов (y)


In [ ]:
wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Classes: {len(np.unique(y))}")


## Создание Dataset

`Dataset` - класс PyTorch для работы с данными. Должен реализовывать:
- `__init__` - инициализация с данными
- `__len__` - возвращает размер датасета
- `__getitem__` - возвращает один пример по индексу (пара X, y)


In [ ]:
class WineDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


## Создание DataLoader

`DataLoader` - загрузчик данных, который:
- Автоматически разбивает данные на батчи заданного размера
- `shuffle=True` для train - перемешивает данные каждую эпоху
- `shuffle=False` для test - сохраняет порядок примеров
- Позволяет эффективно итерироваться по данным в цикле обучения


In [ ]:
train_dataset = WineDataset(X_train, y_train)
test_dataset = WineDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


## Определение модели

Создаем класс нейронной сети, наследуясь от `nn.Module`:
- `__init__` - определяем архитектуру (слои сети)
  - Динамически создаем слои на основе списка `hidden_sizes`
  - Каждый скрытый слой: Linear → ReLU
  - Выходной слой: Linear без активации (логиты для CrossEntropyLoss)
- `forward` - определяем прямой проход (как данные проходят через сеть)


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(NeuralNet, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, num_classes))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)


## Инициализация компонентов обучения

Настраиваем все необходимое для обучения:
- `device` - определяем устройство (GPU если доступно, иначе CPU)
- `model` - создаем модель и переносим на устройство
- `criterion` - функция потерь (CrossEntropyLoss для классификации)
- `optimizer` - оптимизатор Adam с learning rate 0.001


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size=13, hidden_sizes=[32, 16], num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)


## Цикл обучения (Training Loop)

Основной цикл обучения по эпохам:

**Фаза обучения:**
1. `model.train()` - переводим модель в режим обучения
2. Проходим по батчам из train_loader
3. Переносим данные на устройство
4. Forward pass - получаем предсказания
5. Вычисляем loss
6. `optimizer.zero_grad()` - обнуляем градиенты
7. `loss.backward()` - вычисляем градиенты (backpropagation)
8. `optimizer.step()` - обновляем веса

**Фаза валидации:**
1. `model.eval()` - переводим модель в режим оценки
2. `torch.no_grad()` - отключаем вычисление градиентов (экономим память)
3. Получаем предсказания на test данных
4. Вычисляем accuracy


In [ ]:
num_epochs = 100
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    train_losses.append(epoch_loss / len(train_loader))
    
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y_batch.numpy())
        
        test_acc = accuracy_score(all_labels, all_preds)
        test_accuracies.append(test_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_losses[-1]:.4f}, Test Acc: {test_acc:.4f}')


## Визуализация результатов обучения

Строим графики для анализа процесса обучения:
- **Loss** - показывает как уменьшается ошибка на обучающей выборке
- **Accuracy** - показывает как растет точность на тестовой выборке

Это помогает понять:
- Обучается ли модель (loss должен падать)
- Есть ли переобучение (если test accuracy падает при росте train accuracy)
- Нужно ли больше эпох или модель уже сошлась


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(train_losses)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True)

axes[1].plot(test_accuracies)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Test Accuracy')
axes[1].grid(True)

plt.tight_layout()
plt.show()


## Финальная оценка модели

Проводим финальную оценку обученной модели:
- Получаем предсказания на всей тестовой выборке
- Выводим общую точность
- `classification_report` показывает детальные метрики для каждого класса:
  - **precision** - точность (какая доля предсказанных как класс X действительно X)
  - **recall** - полнота (какую долю всех X мы нашли)
  - **f1-score** - гармоническое среднее precision и recall
  - **support** - количество примеров каждого класса


In [ ]:
model.eval()
with torch.no_grad():
    X_test_device = X_test.to(device)
    outputs = model(X_test_device)
    _, predicted = torch.max(outputs, 1)
    predicted = predicted.cpu().numpy()
    
print(f"Final Test Accuracy: {accuracy_score(y_test, predicted):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, predicted, target_names=wine.target_names))
